In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

train_df = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')
sub_df = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')

train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.drop('Id', axis=1, inplace=True)
test_df.drop('Id', axis=1, inplace=True)

In [ ]:
train_df['Soil_Type7'].isnull().sum()

In [ ]:
train_df.Soil_Type7.value_counts()

In [ ]:
train_df.Soil_Type1.value_counts()

In [ ]:
train_df.Soil_Type2.value_counts()

In [ ]:
soil_types = [x for x in train_df.columns if x.startswith('Soil_Type')]
print(len(soil_types))
summation = 0
for i in soil_types:
    summation += train_df[i][train_df[i] == 1].count()
#     print(train_df[i][train_df[i] == 1].count())
    print(train_df[i].value_counts())
mean_ones = summation / (len(soil_types) - 2)
print(mean_ones)

In [ ]:
train_df['Soil_Type7'] = train_df['Soil_Type17']
train_df['Soil_Type15'] = train_df['Soil_Type29']
# cols = ['Soil_Type7', 'Soil_Type15']
# train_df.drop(cols, axis=1, inplace=True)
# test_df.drop(cols, axis=1, inplace=True)

In [ ]:
idx = train_df[train_df['Cover_Type'] == 5].index
# train_df.drop(idx, axis=0, inplace=True)

In [ ]:
train_df.Cover_Type[idx]

In [ ]:
train_df.Cover_Type.value_counts()

In [ ]:
train_df.Cover_Type[idx] = 4
train_df.Cover_Type[idx]

In [ ]:
new_names = {
    'Horizontal_Distance_To_Hydrology': 'x_dist_to_hydrlg',
    'Vertical_Distance_To_Hydrology': 'y_dist_to_hydrlg',
    'Horizontal_Distance_To_Roadways': 'x_dist_to_rdways',
    'Horizontal_Distance_To_Fire_Points': 'x_dist_to_firpnts'
}

train_df.rename(new_names, axis=1, inplace=True)
test_df.rename(new_names, axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
train_df['Cover_Type'] = encoder.fit_transform(train_df['Cover_Type'])


In [ ]:
train_df['Aspect'][train_df['Aspect'] < 0] += 360
train_df['Aspect'][train_df['Aspect'] > 359] -= 360

test_df['Aspect'][test_df['Aspect'] < 0] += 360
test_df['Aspect'][test_df['Aspect'] > 359] -= 360


In [ ]:
train_df['mnhtn_dist_to_hydrlg'] = np.abs(train_df['x_dist_to_hydrlg']) + np.abs(train_df['y_dist_to_hydrlg'])
test_df['mnhtn_dist_to_hydrlg'] = np.abs(test_df['x_dist_to_hydrlg']) + np.abs(test_df['y_dist_to_hydrlg'])

train_df['eucldn_dist_to hydrlg'] = (train_df['x_dist_to_hydrlg'] ** 2 + train_df['y_dist_to_hydrlg'] ** 2) ** 0.5
test_df['eucldn_dist_to hydrlg'] = (test_df['x_dist_to_hydrlg'] ** 2 + test_df['y_dist_to_hydrlg'] ** 2) ** 0.5

In [ ]:
soil_features = [x for x in train_df.columns if x.startswith('Soil_Type')]
train_df['soil_type_count'] = train_df[soil_features].sum(axis=1)
test_df['soil_type_count'] = test_df[soil_features].sum(axis=1)

wilderness_features = [x for x in train_df.columns if x.startswith('Wilderness_Area')]
train_df['wilderness_area_count'] = train_df[wilderness_features].sum(axis=1)
test_df['wilderness_area_count'] = test_df[wilderness_features].sum(axis=1)

In [ ]:
train_df.loc[train_df['Hillshade_9am'] < 0, 'Hillshade_9am'] = 0
test_df.loc[test_df['Hillshade_9am'] < 0, 'Hillshade_9am'] = 0

train_df.loc[train_df['Hillshade_Noon'] < 0, 'Hillshade_Noon'] = 0
test_df.loc[test_df['Hillshade_Noon'] < 0, 'Hillshade_Noon'] = 0

train_df.loc[train_df['Hillshade_3pm'] < 0, 'Hillshade_3pm'] = 0
test_df.loc[test_df['Hillshade_3pm'] < 0, 'Hillshade_3pm'] = 0

train_df.loc[train_df['Hillshade_9am'] > 255, 'Hillshade_9am'] = 255
test_df.loc[test_df['Hillshade_9am'] > 255, 'Hillshade_9am'] = 255

train_df.loc[train_df['Hillshade_Noon'] > 255, 'Hillshade_Noon'] = 255
test_df.loc[test_df['Hillshade_Noon'] > 255, 'Hillshade_Noon'] = 255

train_df.loc[train_df['Hillshade_3pm'] > 255, 'Hillshade_3pm'] = 255
test_df.loc[test_df['Hillshade_3pm'] > 255, 'Hillshade_3pm'] = 255

In [ ]:
train_df

In [ ]:
from sklearn.preprocessing import RobustScaler

cols = [
    'Elevation',
    'Aspect',
    'mnhtn_dist_to_hydrlg',
    'eucldn_dist_to hydrlg',
    'soil_type_count',
    'wilderness_area_count',
    'Slope',
    'x_dist_to_hydrlg',
    'y_dist_to_hydrlg',
    'x_dist_to_rdways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Hillshade_3pm',
    'x_dist_to_firpnts',
    'soil_type_count',
    'wilderness_area_count',
]

scaler = RobustScaler()
train_df[cols] = scaler.fit_transform(train_df[cols])
test_df[cols] = scaler.transform(test_df[cols])

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    
    for col in df.columns:
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
                    
    end_mem = df.memory_usage().sum() / 1024 ** 2
    
    if verbose:
        print('Memory usage decreased to {:5.2f} Mb ({:.1f} % reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
        
    return df

In [ ]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

INPUT_SHAPE = test_df.shape[1:]
NUM_CLASSES = train_df["Cover_Type"].nunique()

def build_model():
    model = Sequential([
        Dense(units=300, kernel_initializer='lecun_normal', activation='selu', input_shape=INPUT_SHAPE),
        BatchNormalization(),
        Dense(units=200, kernel_initializer='lecun_normal', activation='selu'),
        BatchNormalization(),
        Dense(units=100, kernel_initializer='lecun_normal', activation='selu'),
        BatchNormalization(),
        Dense(units=50, kernel_initializer='lecun_normal', activation='selu'),
        BatchNormalization(),
        Dense(units=NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    restore_best_weights=True
)

callbacks = [reduce_lr, early_stop]

In [ ]:
build_model().summary()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(
    build_model(),
    show_shapes=True,
    show_layer_names=True
)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

X = train_df.drop('Cover_Type', axis=1).values
y = train_df['Cover_Type'].values

del train_df

FOLDS = 20
EPOCHS = 200
BATCH_SIZE = 2048

test_preds = np.zeros((1, 1))
scores = []

cv = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model = build_model()
    model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=False
    )
    
    y_pred = np.argmax(model.predict(X_val), axis=1)
    score = accuracy_score(y_val, y_pred)
    scores.append(score)
    
#     test_preds += model.predict(test_df)
    test_preds = test_preds + model.predict(test_df)
    print(f'FOLD {fold} ACCURACY {score}')
    
print()
print(f'MEAN ACCURACY {np.mean(scores)}')

In [ ]:
test_preds = np.argmax(test_preds, axis=1)
test_preds = encoder.inverse_transform(test_preds)

sub_df['Cover_Type'] = test_preds
sub_df

In [ ]:
sub_df.to_csv('submission_04.csv', index=False)